# Degree Centrality

In this notebook, we will compute the degree centrality for  vertices in our test database using cuGraph and NetworkX. The NetworkX and cuGraph processes will be interleaved so that each step can be compared.

| Author Credit |    Date    |  Update          | cuGraph Version |  Test Hardware |
| --------------|------------|------------------|-----------------|----------------|
| Don Acosta    | 07/05/2022 | created | 22.08 nightly   | DGX Tesla V100 CUDA 11.5

## Introduction
Degree centrality is the simplest measure of the relative importance based on counting the connections with each vertex. Vertices with the most connections are the most central by this measure.

See [Degree Centrality on Wikipedia](https://en.wikipedia.org/wiki/Degree_centrality) for more details on the algorithm.



Degree centrality of a vertex 𝑣 is the sum of the edges incident on that node.

<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/62c50cbf5f6cbe0842fe58fca63deb0f0772a829" />


To compute the Degree centrality scores for a graph in cuGraph we use:<br>
__df_v = cugraph.degree_centrality(G)__
    G: cugraph.Graph object
   

Returns:

    df: a cudf.DataFrame object with two columns:
        df['vertex']: The vertex identifier for the vertex
        df['degree_centrality']: The degree centrality score for the vertex


### _NOTICE_
cuGraph does not currently support the ‘endpoints’ and ‘weight’ parameters as seen in the corresponding networkX call. 

#### Some notes about vertex IDs...
* cuGraph will automatically renumber graphs to an internal format consisting of a contiguous series of integers starting from 0, and convert back to the original IDs when returning data to the caller. If the vertex IDs of the data are already a contiguous series of integers starting from 0, the auto-renumbering step can be skipped for faster graph creation times.
  * To skip auto-renumbering, set the `renumber` boolean arg to `False` when calling the appropriate graph creation API (eg. `G.from_cudf_edgelist(gdf_r, source='src', destination='dst', renumber=False)`).
  * For more advanced renumbering support, see the examples in `structure/renumber.ipynb` and `structure/renumber-2.ipynb`


### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


![Karate Club](../../img/zachary_black_lines.png)


Because the test data has vertex IDs starting at 1, the auto-renumber feature of cuGraph (mentioned above) will be used so the starting vertex ID is zero for maximum efficiency. The resulting data will then be auto-unrenumbered, making the entire renumbering process transparent to users.


### Prep

In [ ]:
# Import needed libraries
import cugraph
import cudf

In [ ]:
# NetworkX libraries
import networkx as nx

### Some Prep

In [ ]:
# Define the path to the test data  
datafile='../../data/karate-data.csv'

### Read in the data - GPU
cuGraph depends on cuDF for data loading and the initial Dataframe creation

The data file contains an edge list, which represents the connection of a vertex to another.  The `source` to `destination` pairs is in what is known as Coordinate Format (COO).  In this test case, the data is just two columns.  However a third, `weight`, column is also possible

In [ ]:
gdf = cudf.read_csv(datafile, delimiter='\t', names=['src', 'dst'], dtype=['int32', 'int32'] )

### Create a Graph 

In [ ]:
# create a Graph using the source (src) and destination (dst) vertex pairs from the Dataframe 
G = cugraph.Graph()
G.from_cudf_edgelist(gdf, source='src', destination='dst')

### Call the Degree Centrality algorithm

In [ ]:
# Call cugraph.degree_centrality 
vertex_bc = cugraph.degree_centrality(G)

_It was that easy!_  

----

Let's now look at the results

In [ ]:
# Find the most important vertex using the scores
def print_top_scores(_df, txt) :
    m = _df['degree_centrality'].max()
    _d = _df.query('degree_centrality == @m')
    print(txt)
    print(_d)
    print()
        

In [ ]:
print_top_scores(vertex_bc, "top degree centrality score")

In [ ]:
# let's sort the data and look at the top 5 vertices
vertex_bc.sort_values(by='degree_centrality', ascending=False).head(5)

---

## Now compute using NetworkX

In [ ]:
# Read the data, this also created a NetworkX Graph 
file = open(datafile, 'rb')
Gnx = nx.read_edgelist(file)

In [ ]:
dc_nx_vert = nx.degree_centrality(Gnx)

In [ ]:
dc_nx_sv = sorted(((value, key) for (key,value) in dc_nx_vert.items()), reverse=True)
dc_nx_sv[:5]

As mentioned, the scores are different but the ranking is the same.

___
Copyright (c) 2022, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___